In [ ]:
import re

In [12]:
# Makes the training data out of the whole book (book.txt)
# Project -> Pr/Project and so on
# saves it into "book_processed_train.txt"

def prepare_text_for_training(text):
    # Split text int owords
    words = text.split()
    
    # Clean each word and bring it into form Pr/Project
    processed_words = []
    for word in words:
        clean_word = re.sub('[^a-zA-Z]', '', word) #only alphabetical
        if len(clean_word) > 1:
            processed_word = f"{clean_word[:2]}/{clean_word}"
        elif len(clean_word) == 1:
            processed_word = f"{clean_word}/{clean_word}"
        if processed_word:  # only if existend
            processed_words.append(processed_word)
    
    processed_words += "\n"
    return processed_words

# Do it
# Read
with open('book.txt', 'r') as file:
    text = file.read()

#Process
processed_words = prepare_text_for_training(text)

# Write
with open('book_processed_train.txt', 'w') as file:
    file.write('\n'.join(processed_words))


In [17]:
# Prepares the test text that i took out of the book
# Removes any non alphabetical characters and makes 2-grams of the words
# for testing

def generate2gramText(input_file, output_file):
    # Read test input
    with open(input_file, 'r') as file:
        text = file.read()

    words = text.split()
    
    processed_words = []
    for word in words:
        clean_word = re.sub('[^a-zA-Z]', '', word)
        if clean_word:
            processed_words.append(clean_word[:2])
    
    # Join the processed words with a space
    result = ' '.join(processed_words)
    result += "\n"
    
    # Write
    with open(output_file, 'w') as file:
        file.write(result)


input_file = 'testing_input.txt'
output_file = 'testing_input_processed_test.txt'

generate2gramText(input_file, output_file)


Processed text has been written to testing_input_processed_test.txt


In [21]:
import subprocess

try:
    result = subprocess.run("train-kytea -full book_processed_train.txt -nows -model train.model", shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(result.stdout.decode('utf-8'))
    print(result.stderr.decode('utf-8'))
except subprocess.CalledProcessError as e:
    print(f"Error occurred: {e.stderr.decode('utf-8')}")


Scanning dictionaries and corpora for vocabulary
Reading corpus from book_processed_train.txt  done (90324 lines)
Building dictionary index done!
Creating tagging features (tag 1) done!
Training local tag classifiers done!
Printing model to train.model done!



In [22]:
try:
    result = subprocess.run("kytea -model train.model -out tags < testing_input_processed_test.txt > test.out", shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(result.stdout.decode('utf-8'))
    print(result.stderr.decode('utf-8'))
except subprocess.CalledProcessError as e:
    print(f"Error occurred: {e.stderr.decode('utf-8')}")

In [24]:
# This compares the test text (ground-truth) to the kytea model result (from the 2grams)

def read_clean_text(input_file):
    with open(input_file, 'r') as file:
        text = file.read()
    
    cleaned_text = re.sub(r'[^a-zA-Z\s]', '', text).lower() # Clean text
    
    words = cleaned_text.split() # Split to words
    return words

def compare_texts(input_file1, input_file2):
    # Clean both result and ground-truth texts
    words1 = read_clean_text(input_file1)
    words2 = read_clean_text(input_file2)

    matched_words = 0
    if len(words1) == len(words2):
        for i in range (0, len(words1)):
            if words1[i] == words2[i]:
                matched_words += 1
    
    percentage = 100 * matched_words/len(words1)
    return percentage


input_file1 = 'testing_input.txt'
input_file2 = 'test.out'

percentage = compare_texts(input_file1, input_file2)
print(f"Correctness: {percentage:.2f}%")


Correctness: 58.17%
